In [1]:
import os
os.environ["LOCAL_BACKEND"] = "1"
os.chdir("/home/kailu/EDSR-PyTorch/code")
import sys
from frameworks.superresolution.train_sr_model import test_SR_benchmark

current path is  ['/home/kailu/EDSR-PyTorch/code', '/home/kailu/EDSR-PyTorch/experiment/other_distill', '/home/kailu/miniconda3/envs/wukailu/lib/python37.zip', '/home/kailu/miniconda3/envs/wukailu/lib/python3.7', '/home/kailu/miniconda3/envs/wukailu/lib/python3.7/lib-dynload', '', '/home/kailu/miniconda3/envs/wukailu/lib/python3.7/site-packages', '/home/kailu/miniconda3/envs/wukailu/lib/python3.7/site-packages/IPython/extensions', '/home/kailu/.ipython']
path for model is  /home/kailu/EDSR-PyTorch/code/model/__init__.py
current backend is  local_backend


In [57]:
config = "/home/kailu/EDSR-PyTorch/code/frameworks/pisr/configs/splainx2/step2.yml"

In [58]:
import torch
from torch import nn
from pdb import set_trace

def quantize(img, rgb_range):
    pixel_range = 255 / rgb_range
    return img.mul(pixel_range).clamp(0, 255).round().div(pixel_range)

class PISR_model(nn.Module):
    def __init__(self, config_path):
        super().__init__()
        import frameworks.pisr.utils.config as config
        import frameworks.pisr.utils.checkpoint as checkpoint
        from frameworks.pisr.models import get_model
        from frameworks.pisr.evaluate import get_test_dataloader
        config = config.load(config_path)
        os.environ["CUDA_VISIBLE_DEVICES"]= str(config.gpu)
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        student_model = get_model(config, 'student').to(device)
        optimizer_s = None
        checkpoint_s = checkpoint.get_initial_checkpoint(config, model_type='student')
        last_epoch_s, step_s = checkpoint.load_checkpoint(student_model, optimizer_s, checkpoint_s, model_type='student')
        self.model = student_model
        self.config = config
        self.old_loader = get_test_dataloader(config)
        
    def forward(self, x):
        student_pred_dict = self.model.forward(LR=x/255)
        pred_hr = student_pred_dict['hr']
        pred_hr = quantize(pred_hr, self.config.data.rgb_range)
#         set_trace()
        return pred_hr * 255

In [59]:
def create_shell(test_model):
    from frameworks.superresolution.SRModel import SR_LightModel
    shell = SR_LightModel({
        'metric': f'psnr_gray_shave_x{test_model.config.data.scale}', 
        'precision': 32, 
        'max_lr': 0.0002, 
        'learning_rate': 0.0002, 
        'lr': 0.0002,
        'weight_decay': 0, 
        'lr_scheduler': 'OneCycLR', 
        'optimizer': 'Adam', 
        'scale': test_model.config.data.scale, 
        'dataset': {
            'workers': 4, 
            'name': 'DIV2K', 
            'scale': test_model.config.data.scale,
            'total_batch_size': 16, 
            'patch_size': 192, 
            'ext': 'sep', 
            'repeat': 20, 
            'test_bz': 1, 
            'batch_size': 16
        }, 
        'rgb_range': 255,
        'seed': 233, 
        'backbone': {
            'arch': 'EDSR_layerwise_sr', 
            'n_feats': 64, 
            'n_resblocks': 16, 
            'simple_tail': 1, 
            'scale': test_model.config.data.scale
        }, 
    })
    shell.model = test_model
    return shell

In [60]:
model = create_shell(PISR_model(config))

*************************
data {'scale': 2, 'n_colors': 3, 'rgb_range': 1, 'num_workers': 3, 'pin_memory': False, 'train': [{'name': 'DIV2K', 'params': {'base_dir': '/data/DIV2K', 'augment': True, 'patch_size': 192, 'data_range': '1-800', 'test_every': 1000}}], 'valid': [{'name': 'DIV2K', 'params': {'base_dir': '/data/DIV2K', 'augment': False, 'patch_size': 192, 'data_range': '801-810'}}], 'test': [{'name': 'Set5', 'params': {'base_dir': '/data/Set5'}}, {'name': 'Set14', 'params': {'base_dir': '/data/Set14'}}, {'name': 'B100', 'params': {'base_dir': '/data/B100'}}, {'name': 'Urban100', 'params': {'base_dir': '/data/Urban100'}}]}
*************************
train {'dir': '../PISR/results/splainnetx2/'}
*************************
teacher_model {'name': 'plainnet_teacher', 'params': {'scale': 2, 'n_colors': 3, 'm': 4, 'k': 1, 'encoder': 'lcscc', 'n_feats': 64, 'num_modules': 19}}
*************************
params {'scale': 2, 'n_colors': 3, 'm': 4, 'k': 1, 'encoder': 'lcscc', 'n_feats': 64, '

In [61]:
test_SR_benchmark(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]


dataset_params: {'name': 'Set5', 'test_only': True, 'patch_size': 192, 'ext': 'sep', 'scale': 2, 'batch_size': 1, 'dataset_mapping': (0, 1, 2), 'dataset_transforms': (0, 1, 1)}
len(dataset): 5
len(dataset): 5
len(dataset): 5


Testing: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/psnr_gray_shave_x2': 37.32267379760742}
--------------------------------------------------------------------------------
--------------> Set5_psnr_gray_shave_x2 = 37.32267379760742 <-------------
dataset_params: {'name': 'Set14', 'test_only': True, 'patch_size': 192, 'ext': 'sep', 'scale': 2, 'batch_size': 1, 'dataset_mapping': (0, 1, 2), 'dataset_transforms': (0, 1, 1)}
len(dataset): 14
len(dataset): 14
len(dataset): 14


...

Testing: 0it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/psnr_gray_shave_x2': 31.70647621154785}
--------------------------------------------------------------------------------
--------------> B100_psnr_gray_shave_x2 = 31.70647621154785 <-------------
dataset_params: {'name': 'Urban100', 'test_only': True, 'patch_size': 192, 'ext': 'sep', 'scale': 2, 'batch_size': 1, 'dataset_mapping': (0, 1, 2), 'dataset_transforms': (0, 1, 1)}
len(dataset): 100
len(dataset): 100
len(dataset): 100


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/psnr_gray_shave_x2': 30.481748580932617}
--------------------------------------------------------------------------------
--------------> Urban100_psnr_gray_shave_x2 = 30.481748580932617 <-------------
